In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Assumes scale images are on your Drive under "Scales"

!ls /content/drive/MyDrive/Images

147863_1_L.tif	 166155_1_L.tif  166511_3_L.tif   167379_6_L.tif   168238_1_L.tif   172829_8_L.tif
148637_1_L.tif	 166164_4_L.tif  166511_4_L.tif   167396_7_L.tif   168240_4_L.tif   172831_4_L.tif
148637_4_L.tif	 166167_2_L.tif  166511_5_L.tif   167440_2_L.tif   168240_5_L.tif   172831_6_L.tif
148637_5_L.tif	 166170_1_L.tif  166511_6_L.tif   167444_8_L.tif   168256_2_L.tif   172846_1_L.tif
148637_8_L.tif	 166183_1_L.tif  166511_7_L.tif   167481_2_L.tif   168256_8_L.tif   172846_6_L.tif
148660_4_L.tif	 166183_2_L.tif  166518_1_L.tif   167481_5_L.tif   168259_1_L.tif   172846_7_L.tif
148690_6_L.tif	 166183_4_L.tif  166518_3_L.tif   167481_6_L.tif   168314_3_LB.tif  172847_6_L.tif
148690_7_L.tif	 166183_5_L.tif  166518_7_L.tif   167481_7_L.tif   169287_5_L.tif   172880_3_L.tif
148815_8_L.tif	 166183_6_L.tif  166544_5_L.tif   167481_8_L.tif   169294_2_L.tif   173032_8_L.tif
158324_1_L.tif	 166183_7_L.tif  166574_1_L.tif   167481_9_L.tif   169336_2_L.tif   173077_7_L.tif
158334_1_L.tif	 1661

In [7]:
import numpy as np
from matplotlib.image import imread
import matplotlib.pyplot as plt

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def scale_edge_detector(x,index, window_size, frequency_factor, orientation):
  if not (orientation == 'LR' or orientation == 'RL'):
    raise Exception("Edge detection orientation must be 'LR' or 'RL'.")

  if window_size % 2 != 1:
    raise Exception("Window size must be an odd number.")

  l_sum = np.sum(x[index - int(window_size / 2): index])
  r_sum = np.sum(x[index + 1: index + int(window_size / 2)])

  if orientation == 'LR':
    if r_sum > l_sum * frequency_factor:
      return True
    else:
      return False
  else:
    if r_sum * frequency_factor > l_sum:
      return True
    else:
      return False

def mask1d(gray_img, kernel_size, threshold=20):
  if kernel_size % 2 != 1:
    raise Exception("Kernel size must be an odd number.")


  mask = np.zeros(gray_img.shape, dtype=np.uint8)
  filter = np.ones(kernel_size) / kernel_size

  for y in range(gray_img.shape[0]):
    delta = np.convolve(gray_img[y,:], filter, mode='same') - gray_img[y,:]
    mask[y, (np.abs(delta) > threshold)] = 255.

  return mask

def mask(gray_img, kernel_size, threshold=20):
  if kernel_size % 2 != 1:
    raise Exception("Kernel size must be an odd number.")

  xd = mask1d(gray_img, 9, 20)
  yd = mask1d(np.transpose(gray_img), 9, 20).transpose()
  mask = xd /2 + yd / 2

  return mask

def denoise(gray_img, kernel_size, percent_activated_threshold=0.4):
  if kernel_size % 2 != 1:
    raise Exception("Kernel size must be an odd number.")

  activation_cutoff = percent_activated_threshold * kernel_size * kernel_size * 255
  denoised_img = gray_img.copy()

  for j in range(int(kernel_size / 2), gray_img.shape[0] - int(kernel_size / 2)):
    for k in range(int(kernel_size / 2), gray_img.shape[1] - int(kernel_size / 2)):
      if gray_img[j, k] > 120:
        if np.sum(gray_img[j:j+kernel_size,k:k+kernel_size]) < activation_cutoff:
          denoised_img[j, k] = 0

  return denoised_img

def detect_scale_contour(gray_img, padding=25):
  contour = []

  for j in range(padding, gray_img.shape[0] - padding):
    k = padding
    k_min = padding

    while k < gray_img.shape[1] - padding:
      if gray_img[j, k] > 0 and scale_edge_detector(gray_img[j,:], k, 25, 5, 'LR'):
        contour.append((j, k))
        k_min = k
        k = gray_img.shape[1] - padding
      k = k + 1

    while k > k_min:
      if gray_img[j, k] > 0 and scale_edge_detector(gray_img[j,:], k, 25, 5, 'RL'):
        contour.append((j, k))
        k = 0
      k = k - 1

  return contour

def dilate_contour(contour, thickness):

  dialated_contour = set()
  for (j, k) in contour:
    for l in range(j - thickness, j + thickness):
      for m in range(k - thickness, k + thickness):
        dialated_contour.add((l, m))

  return list(dialated_contour)

def draw_box(gray_img, top_left_point, bottom_right_point, color=255):
  j1, k1 = top_left_point
  j2, k2 = bottom_right_point

  gray_img[j1,k1:k2+1] = color
  gray_img[j2,k1:k2+1] = color

def draw_contour(size, contour, color=255):

  img = np.zeros(size, dtype=np.uint8)
  for (j, k) in contour:
    img[j, k] = color

  return img

def show_image_and_contour(file):
  img = imread(file)

  print(file)
  gray = rgb2gray(img)
  plt.imshow(gray, cmap=plt.get_cmap('gray'))
  plt.show()

  masked_image = mask(gray, 9, 20)
  denoised_mask = denoise(masked_image, 101, .025)

  contour = detect_scale_contour(denoised_mask, 100)
  dilated_contour = dilate_contour(contour, 5)
  drawn_contour = draw_contour(denoised_mask.shape, dilated_contour)

  plt.imshow(np.abs(drawn_contour - 255), cmap=plt.get_cmap('gray'))
  plt.show()

In [9]:
import glob

for image_file in glob.glob("/content/drive/MyDrive/Images/*tif"):
  show_image_and_contour(image_file)

Output hidden; open in https://colab.research.google.com to view.